### Building an 8-puzzle solver

In [1]:
%cd C:\Users\U\Artificial Intelligence\data
#To change the directory to data for simpleai's dependencies
    
from data.simpleai.search import astar, SearchProblem

C:\Users\U\Artificial Intelligence\data


In [2]:
class PuzzleSolver(SearchProblem):
    #Class containing methods to solve the puzzle
    def actions(self, cur_state):
        #Get the list of possible numbers that can be moved into the empty space
        rows = string_to_list(cur_state)
        row_empty, col_empty = get_location(rows, 'e')
        
        #Check the location of the empty space and create the new action
        actions = []
        if row_empty > 0:
            actions.append(rows[row_empty - 1][col_empty])
        if row_empty < 2:
            actions.append(rows[row_empty + 1][col_empty])
            
        if col_empty > 0:
            actions.append(rows[row_empty][col_empty - 1])
        if col_empty < 2:
            actions.append(rows[row_empty][col_empty + 1])
            
        return actions
    
    def result(self, state, action):
        rows = string_to_list(state)
        row_empty, col_empty = get_location(rows, 'e')
        row_new, col_new = get_location(rows, action)
        
        rows[row_empty][col_empty], rows[row_new][col_new] = rows[row_new][col_new], rows[row_empty][col_empty]
        
        return list_to_string(rows)
    
    def is_goal(self, state):
        #Check if the goal has been reached
        return state == GOAL
    
    def heuristic(self, state):
        #Returns an estimate of the distance from a state to the goal using the manhattan distance
        rows = string_to_list(state)
        
        distance = 0
        
        for number in '12345678e':
            row_new, col_new = get_location(rows, number)
            row_new_goal, col_new_goal = goal_positions[number]
            
            distance += abs(row_new - row_new_goal) + abs(col_new - col_new_goal)
            
        return distance

In [3]:
def list_to_string(input_list):
    return '\n'.join(['-'.join(x) for x in input_list])

def string_to_list(input_string):
    return [x.split('-') for x in input_string.split('\n')]

def get_location(rows, input_element):
    #Find the 2D location of the input element
    for i, row in enumerate(rows):
        for j, item in enumerate(row):
            if item == input_element:
                return i, j

In [4]:
GOAL = """1-2-3
4-5-6
7-8-e"""

INITIAL = """1-e-2
6-3-4
7-5-8"""

In [5]:
#Create a cache for the goal position of each piece
goal_positions = {}
rows_goal = string_to_list(GOAL)

for number in '12345678e':
    goal_positions[number] = get_location(rows_goal, number)

In [6]:
result = astar(PuzzleSolver(INITIAL))

In [7]:
#Print the results
for i, (action, state) in enumerate(result.path()):
    if action == None:
        print('Initial Configuration')
    elif i == len(result.path()) - 1:
        print(f'After moving {action} into the empty space. Goal achieved!')
    else:
        print(f'After moving {action} into the empty space')
        
    print(state,'\n')

Initial Configuration
1-e-2
6-3-4
7-5-8 

After moving 2 into the empty space
1-2-e
6-3-4
7-5-8 

After moving 4 into the empty space
1-2-4
6-3-e
7-5-8 

After moving 3 into the empty space
1-2-4
6-e-3
7-5-8 

After moving 6 into the empty space
1-2-4
e-6-3
7-5-8 

After moving 1 into the empty space
e-2-4
1-6-3
7-5-8 

After moving 2 into the empty space
2-e-4
1-6-3
7-5-8 

After moving 4 into the empty space
2-4-e
1-6-3
7-5-8 

After moving 3 into the empty space
2-4-3
1-6-e
7-5-8 

After moving 6 into the empty space
2-4-3
1-e-6
7-5-8 

After moving 4 into the empty space
2-e-3
1-4-6
7-5-8 

After moving 2 into the empty space
e-2-3
1-4-6
7-5-8 

After moving 1 into the empty space
1-2-3
e-4-6
7-5-8 

After moving 4 into the empty space
1-2-3
4-e-6
7-5-8 

After moving 5 into the empty space
1-2-3
4-5-6
7-e-8 

After moving 8 into the empty space. Goal achieved!
1-2-3
4-5-6
7-8-e 



### Building a maze solver

In [8]:
from math import sqrt, fsum, cos, sin

In [9]:
class MazeSolver(SearchProblem):
    def __init__(self, board):
        self.board = board
        self.goal = (0, 0)
    
        for y in range(len(self.board)):
            for x in range(len(self.board[y])):
                if self.board[y][x].lower() == 'o':
                    self.initial = (x, y)
                elif self.board[y][x].lower() == 'x':
                    self.goal = (x, y)
                    
        super(MazeSolver, self).__init__(initial_state = self.initial)
        
    def actions(self, state):
        #Define the actions that takes action to arrive at the solution
        actions = []
        
        for action in COSTS.keys():
            newx, newy = self.result(state, action)
            if self.board[newy][newx] != '#':
                actions.append(action)
        
        return actions
    
    def result(self, state, action):
        #Update the state based on action
        x, y = state
        
        if action.count('up'):
            y -= 1
        if action.count('down'):
            y += 1
        if action.count('left'):
            x -= 1
        if action.count('right'):
            x += 1
            
        new_state = (x, y)
        
        return new_state
    
    def is_goal(self, state):
        return state == self.goal
    
    def cost(self, state, action, state2):
        return COSTS[action]
    
    def heuristic(self, state):
        x, y = state
        gx, gy = self.goal
        
        return sqrt((x - gx) ** 2 + (y - gy) ** 2)

In [21]:
MAP = """
##############################
#         #              #   #
# ####    ########       #   #
#  o #    #              #   #
#    ###     #####  ######   #
#      #   ###   #           #
#      #     #   #  #  #   ###
#     #####    #    #  # x   #
#              #       #     #
##############################
"""

In [22]:
#Convert map to a list
print(MAP)
MAP = [list(x) for x in MAP.split('\n') if x]


##############################
#         #              #   #
# ####    ########       #   #
#  o #    #              #   #
#    ###     #####  ######   #
#      #   ###   #           #
#      #     #   #  #  #   ###
#     #####    #    #  # x   #
#              #       #     #
##############################



In [23]:
#Define the cost of moving around the map, a diagonal move is more expensive
cost_regular = 1.0
cost_diagonal = 1.7

In [24]:
COSTS = {
    'up': cost_regular, 
    'down': cost_regular, 
    'left': cost_regular, 
    'right': cost_regular, 
    'up left': cost_diagonal, 
    'up right': cost_diagonal, 
    'down left': cost_diagonal, 
    'down right': cost_diagonal
}

In [25]:
problem = MazeSolver(MAP)

In [26]:
result = astar(problem, graph_search = True)

In [28]:
path = [x[1] for x in result.path()]

In [35]:
#Print the result
#print()
for y in range(len(MAP)):
    for x in range(len(MAP[y])):
        if (x, y) == problem.initial:
            print('o', end = '')
        elif (x, y) == problem.goal:
            print('x', end = '')
        elif (x, y) in path:
            print('.', end = '')
        else:
            print(MAP[y][x], end = '')
    print()


##############################
#         #              #   #
# ####    ########       #   #
#  o #    #              #   #
#   .###     #####  ######   #
#   .  #   ###   #  ....     #
#   .  #     # ..# .#  #.  ###
#    .#####   .# .. #  # x   #
#     ........ #       #     #
##############################
